In [3]:
# Install required packages (run this cell if packages are not installed)
# Uncomment the line below to install:

!pip install langgraph langchain-core langchain-openai langchain-community python-dotenv ipython


^C


  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/879.1 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/879.1 kB 1.0 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/879.1 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 879.1/879.1 kB 973.1 kB/s  0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 982.3

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\SPYROSDP\\Work\\Learning\\AI_Agents\\AI_Agents_in_LangGraph\\.venv\\Lib\\site-packages\\aiohttp\\client.py'
Check the permissions.


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Lesson 2 : LangGraph Components

In [6]:
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI #could use gpt or other LLM
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
#Search tool from langchain community

tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name) #name used by agent to use tool

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [10]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add] #will be appending to the list - hence the reducer

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [11]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools) #letting model know it has tools

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0 #true if any tool calls

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]} #adds to state due to reducer

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls #could be a list
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [15]:
#from IPython.display import Image

#Image(abot.graph.get_graph().draw_png())

In [16]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_35kUgx5r8s7WVMxAJ3CdAlfa', 'type': 'tool_call'}
Back to the model!


In [ ]:
result #simply is a list of messages i.e. final state 

{'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CbBC3XZrIw6u9HPTR72trgp1zTiGL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--76cac473-94c4-42de-b091-13f7031d9a6e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_35kUgx5r8s7WVMxAJ3CdAlfa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 21, 'total_tokens': 174, 'input_toke

In [18]:
result['messages'][-1].content

'I found some information on the weather in San Francisco. The historical weather data shows that the highest temperature recorded was 76.5℉ on October 29, and the lowest temperature was 50.8℉ on October 28. There was a total of 0.071" of precipitation during this period, with the most precipitation falling on November 5.\n\nIf you need more current weather information, let me know!'

In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")] #does parallel function/ tool calling
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_4XfvogJsT6wky9H9czPRwUxS', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_MPNmaiMbs2pcUhXouZHJhsM9', 'type': 'tool_call'}
Back to the model!


In [20]:
result['messages'][-1].content

'The weather information for San Francisco shows that historical data indicates temperatures ranging from 46°F to 57°F in December with some rainy days. For Los Angeles, the temperature range in December is typically 11°C to 19°C with an average of 5 days of rain.'

In [ ]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

# sequential function calling needed for next tool use

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_nZhsmg6CIzjWRLcRQqdr2FbB', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Philadelphia Eagles headquarters location'}, 'id': 'call_SAhXAMv9hVEAicw0dGuFpfkP', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Pennsylvania 2024'}, 'id': 'call_vRp3krPU4PgB7tZlyW4JPXb6', 'type': 'tool_call'}
Back to the model!


In [22]:
print(result['messages'][-1].content)

1. **Who won the Super Bowl in 2024?**
   - The Philadelphia Eagles won the Super Bowl in 2024.

2. **In what state is the winning team's headquarters located?**
   - The headquarters of the Philadelphia Eagles is located in Pennsylvania.

3. **What is the GDP of that state?**
   - The real GDP of Pennsylvania in 2024 was approximately $818.5 billion.
